# Multi-period Dispatch Simulation

Multi-period dispatch economic dispatch (ED) and unit commitment (UC) is also available.

In this case, we will show a 24-hour ED simulation.

In [1]:
import ams

import datetime

In [2]:
print("Last run time:", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

print(f'ams:{ams.__version__}')

Last run time: 2024-03-02 16:59:25
ams:0.9.1


In [3]:
ams.config_logger(stream_level=20)

## Load Case

In [4]:
sp = ams.load(ams.get_case('5bus/pjm5bus_demo.xlsx'),
              setup=True,
              no_output=True,)

Parsing input file "/Users/jinningwang/Documents/work/mambaforge/envs/amsre/lib/python3.9/site-packages/ams/cases/5bus/pjm5bus_demo.xlsx"...
Input file parsed in 0.1031 seconds.
Zero line rates detacted in rate_a, rate_b, rate_c, adjusted to 999.
If expect a line outage, please set 'u' to 0.
System set up in 0.0019 seconds.


## Reginonal Design

The disaptch models in AMS has develoepd with regional structure, and it can be inspected in device ``Region``.

In [5]:
sp.Region.as_df()

,idx,u,name
uid,,,
0,ZONE_1,1.0,ZONE 1
1,ZONE_2,1.0,ZONE 2


In device ``Bus``, the Param ``zone`` indicates the zone of the bus.
Correspondingly, the region of generator and load are determined by the bus they connected.

In [6]:
sp.Bus.as_df()

,idx,u,name,Vn,vmax,vmin,v0,a0,xcoord,ycoord,area,zone,owner
uid,,,,,,,,,,,,,
0,Bus_1,1.0,A,230.0,1.1,0.9,1.0,0.0,0,0,1,ZONE_1,None
1,Bus_2,1.0,B,230.0,1.1,0.9,1.0,0.0,0,0,1,ZONE_1,None
2,Bus_3,1.0,C,230.0,1.1,0.9,1.0,0.0,0,0,2,ZONE_1,None
3,Bus_4,1.0,D,230.0,1.1,0.9,1.0,0.0,0,0,2,ZONE_1,None
4,Bus_5,1.0,E,230.0,1.1,0.9,1.0,0.0,0,0,3,ZONE_1,None


## Multi-period Dispatch Base

In AMS, multi-period dispatch involves devices in group ``Horizon``.
This group is developed to provide time-series data for multi-period dispatch.

In [7]:
sp.Horizon.models

OrderedDict([('TimeSlot', TimeSlot (0 devices) at 0x137ec7610),
             ('EDTSlot', EDTSlot (6 devices) at 0x137ed50d0),
             ('UCTSlot', UCTSlot (6 devices) at 0x137ed54f0)])

We can get the idx of StaticGens.

In [8]:
sp.StaticGen.get_idx()

['PV_1', 'PV_3', 'PV_5', 'Slack_4']

In ``EDTSlot``, Param ``sd`` refers the load factors of each region in each time slot, and Param ``ug`` represents the generator commitment status in each time slot.

To be more specific, EDT1 has ``sd=0.0793,0.0``, which means the load factor of region 1 is 0.0793 in the first time slot, and 0.0 in the second time slot.

Next, EDT1 has ``ug=1,1,1,1``, and it means the commitment status of generator PV_1, PV_3, PV_5, and Slack_4 are all online.

In [9]:
sp.EDTSlot.as_df()

,idx,u,name,sd,ug
uid,,,,,
0,EDT1,1.0,EDT1,"0.793,0.0","1,1,1,1"
1,EDT2,1.0,EDT2,"0.756,0.0","1,1,1,1"
2,EDT3,1.0,EDT3,"0.723,0.0","1,1,1,1"
3,EDT4,1.0,EDT4,"0.708,0.0","1,1,1,1"
4,EDT5,1.0,EDT5,"0.7,0.0","1,1,1,1"
5,EDT6,1.0,EDT6,"0.706,0.0","1,1,1,1"


## Solve and Result

In [10]:
sp.ED.init()

Routine <ED> initialized in 0.0178 seconds.


True

In [11]:
sp.ED.run(solver='ECOS')

ED solved as optimal in 0.0230 seconds, converged after 11 iterations using solver ECOS.


True

All decision variables are collected in the dict ``vars``.

In [12]:
sp.ED.vars

OrderedDict([('pg', Var: StaticGen.pg),
             ('aBus', Var: Bus.aBus),
             ('plf', Var: Line.plf),
             ('pru', Var: StaticGen.pru),
             ('prd', Var: StaticGen.prd),
             ('prs', Var: StaticGen.prs)])

As we can see, the generator output ``pg`` is a 2D array, and the first dimension is the generator index, and the second dimension is the time slot.

In [13]:
sp.ED.pg.v

array([[2.1 , 2.1 , 2.1 , 2.1 , 2.1 , 2.1 ],
       [3.23, 2.86, 2.53, 2.38, 2.3 , 2.36],
       [0.6 , 0.6 , 0.6 , 0.6 , 0.6 , 0.6 ],
       [2.  , 2.  , 2.  , 2.  , 2.  , 2.  ]])

Partial results can be accessed with desired time slot.
In the retrieved result, the first dimension is the generator index, and the second dimension is the time slot.

In [14]:
sp.ED.get(src='pg', attr='v', idx='PV_1', horizon=['EDT1'])

array([2.1])

Or, get multiple variables in mutliple time slots.

In [15]:
sp.ED.get(src='pg', attr='v', idx=['PV_1', 'PV_3'], horizon=['EDT1', 'EDT2', 'EDT3'])

array([[2.1 , 2.1 , 2.1 ],
       [3.23, 2.86, 2.53]])